In [1]:
import warnings
warnings.simplefilter("ignore")

## Escalamiento de los datos

In [2]:
%matplotlib inline
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

<div class="alert alert-success">
  Diversos algoritmos son sensibles a la escala en la que viene cada feature. **Re-escalarlos** puede traer significativas mejoras de rendimiento.
</div>

Existen distintas estrategias de escalamiento de tus features, pero **la más común es la estandarización** donde convertimos la variable para que la distribución de esta siga una distribución que es Gaussiana de media 0 y de desviación estandar 1.

In [3]:
from sklearn.model_selection import train_test_split

X = pd.read_csv('data/X.csv')
y = X['worldwide_gross']
X = X.drop('worldwide_gross',axis=1)
X_train, X_test, y_train, y_test = train_test_split(X,y)

In [4]:
from sklearn.preprocessing import StandardScaler

scaler = StandardScaler()
scaler.fit(X_train)

StandardScaler()

In [5]:
scaler.mean_

array([3.41170986e+07, 2.00237243e+03, 2.13416865e+00, 1.08424827e+02,
       1.03713018e+04, 4.23844612e+07, 6.45389864e+00])

In [6]:
scaler.scale_

array([4.19246700e+07, 1.18822379e+01, 7.61094042e-01, 2.29010227e+01,
       1.90046683e+04, 2.31610288e+08, 1.06542450e+00])

In [7]:
X.values

array([[4.25000000e+08, 2.00900000e+03, 1.78000000e+00, ...,
        4.83400000e+03, 2.37000000e+08, 7.90000000e+00],
       [3.06000000e+08, 2.00213073e+03, 2.12697615e+00, ...,
        1.43000000e+02, 4.04553863e+07, 7.10000000e+00],
       [3.00000000e+08, 2.00700000e+03, 2.35000000e+00, ...,
        4.83500000e+04, 3.00000000e+08, 7.10000000e+00],
       ...,
       [7.00000000e+03, 2.00500000e+03, 2.12697615e+00, ...,
        9.30000000e+01, 3.25000000e+03, 7.80000000e+00],
       [3.96700000e+03, 2.01200000e+03, 2.35000000e+00, ...,
        2.38600000e+03, 4.04553863e+07, 6.30000000e+00],
       [1.10000000e+03, 2.00400000e+03, 1.85000000e+00, ...,
        1.63000000e+02, 1.10000000e+03, 6.60000000e+00]])

In [8]:
scaler.transform(X_train)

array([[-0.57524838, -0.0313438 , -0.37336864, ..., -0.44017089,
        -0.13982307, -2.67864933],
       [-0.40828225, -0.11550303, -0.37336864, ..., -0.36976714,
        -0.03620073, -1.45847841],
       [-0.26516842, -1.29373225,  0.2835804 , ..., -0.42180698,
        -0.0836943 , -3.42952374],
       ...,
       [ 1.21367447, -0.0313438 , -0.37336864, ..., -0.46890068,
         0.18399674, -0.8953226 ],
       [-0.57524838,  0.47361159, -0.37336864, ..., -0.39802335,
        -0.14414067,  0.23098902],
       [ 1.09441294,  1.0627262 ,  0.2835804 , ...,  0.3446889 ,
         0.20558473, -0.42602609]])

In [9]:
X_train_scaled, X_test_scaled = (scaler.transform(X_train), scaler.transform(X_test))

In [10]:
from sklearn.linear_model import Lasso

model = Lasso()
model_scaled = Lasso()

model.fit(X_train,y_train)
model_scaled.fit(X_train_scaled,y_train)

Lasso()

In [11]:
print(model.score(X_test,y_test))
print(model_scaled.score(X_test_scaled,y_test))

0.5358767537636218
0.5358767534278537


<div class="alert alert-success">
  Los modelos de regresión no se ven afectados por el escalamiento de las features. Los de clasificación sí. 
</div>

## Simplificar las transformaciones con pipelines

<div class="alert alert-info">
  Para hacer tu código más reproducible, y para evitar tener que aplicar multiples veces una misma transformación te recomendamos utilizar <code> sklearn.pipeline.make_pipeline </code> que permite encadenar transformaciones a tus modelos.
</div>

In [12]:
from sklearn.pipeline import make_pipeline

model_scaled = make_pipeline(StandardScaler(),
                            Lasso())

model_scaled.fit(X_train,y_train)

Pipeline(steps=[('standardscaler', StandardScaler()), ('lasso', Lasso())])

In [13]:
print(model_scaled.score(X_test,y_test))

0.5358767534278537


## Crear nuevas features de forma automática

In [14]:
A = np.arange(6).reshape(3, 2)
A

array([[0, 1],
       [2, 3],
       [4, 5]])

In [15]:
from sklearn.preprocessing import PolynomialFeatures

transformer = PolynomialFeatures(2)
transformer.fit_transform(A)

array([[ 1.,  0.,  1.,  0.,  0.,  1.],
       [ 1.,  2.,  3.,  4.,  6.,  9.],
       [ 1.,  4.,  5., 16., 20., 25.]])

``PolynomialFeatures`` transforma una matriz $(A1,A2)$ a $(1,A1,A2,A1^2,A1\cdot A2,A2^2)$

In [16]:
X.shape

(4104, 7)

In [17]:
transformer = PolynomialFeatures(2)
transformer.fit_transform(X).shape

(4104, 36)

In [18]:
model_poly = make_pipeline(PolynomialFeatures(2),
                          Lasso())
model_poly.fit(X_train,y_train)
model_poly.score(X_test,y_test)

0.5736735836607353

In [19]:
model = Lasso()
model.fit(X_train,y_train)
model.score(X_test,y_test)

0.5358767537636218

## Crear features categóricas

<div class="alert alert-success">
  En terminos de Machine Learning a las features que pueden tomar un número finito de valores se les llama categóricas. Ejemplos para esto són: género, páis, grado académico, etc.
</div>

Un mapeo del tipo $\{Perú, Chile, Colombia, Venezuela\} \rightarrow \{1, 2, 3, 4\}$ tiene el problema de asignarle un ordén a los valores posibles de la categoría. Este orden impacta de distintas maneras los algoritmos de Machine Learning, por ejemplo aquellos que dependen de la topología de $R^n$ y de la función de distancia entre puntos en este espacio, considerarán que ciertas categorías se encuentran más cercanas unas de otras, siendo que esto es generado puramente artificialmente por el encoding, y no por las datos per se. 

Para no introducir información falsa o erronéa en nuestro modelos existen formas más inteligentes de encodear nuestros datos.

_**Encoding one-hot**_

Este encoding consiste en asignarle una columna a cada categoría y rellenarla con 0 y 1 de la forma siguiente:

In [20]:
d = pd.DataFrame([['Chile','Colombia','Colombia','Venezuela'],['hombre','mujer','hombre','mujer']])
d = d.T
d.columns = pd.Index(['pais','genero'])
d

,pais,genero
0,Chile,hombre
1,Colombia,mujer
2,Colombia,hombre
3,Venezuela,mujer


In [21]:
pd.get_dummies(d)

,pais_Chile,pais_Colombia,pais_Venezuela,genero_hombre,genero_mujer
0,1,0,0,1,0
1,0,1,0,0,1
2,0,1,0,1,0
3,0,0,1,0,1


Sklearn también ofrece un objeto OneHotEncoder pero es un poco más díficil de utilizar, así que por criterios pedagogicos hemos elegido ``pd.get_dummies``. Sin embargo el objeto de sklearn tiene la ventaja de ser pipeable, por lo que es bueno considerarlo para ciertos casos de uso.

Cuantas columnas generaríamos con un one-hot encoding de nuestras features categóricas?

In [22]:
movies_obj = pd.read_csv('https://raw.githubusercontent.com/JuanPabloMF/datasets-platzi-course/master/intermediate_results/movies_obj.csv')

In [23]:
movies_obj.apply(pd.Series.nunique).sort_values()

color                2
content_rating      18
language            47
country             65
genres             914
actor_1_name      2097
director_name     2398
actor_2_name      3032
actor_3_name      3521
plot_keywords     4760
movie_title       4917
dtype: int64

Las features más informativas son las del casting. Si embargo haciendo un one-hot encoding de estas estaríamos aumentando la dimensión por 2000 y algo!!

## Encoding Binario

Esta técnica no es canónica por lo que tendremos que buscarla en otra librería. Sin embargo el autor tuvo la buena idea de hacer su API compatible con la de sklearn, así que no tendremos ninguna dificultad en usarla.

$$ Categoria \rightarrow Numero \rightarrow Binario \rightarrow Columnas $$

In [24]:
!pip install category_encoders

In [25]:
categoricals = pd.read_csv('https://raw.githubusercontent.com/JuanPabloMF/datasets-platzi-course/master/intermediate_results/categoricals.csv').set_index('Unnamed: 0')

In [26]:
categoricals.head(2)

,actor_1_name,director_name
Unnamed: 0,,
0,CCH Pounder,James Cameron
1,Doug Walker,Doug Walker


In [27]:
categoricals = categoricals.reset_index(drop=True).fillna(0)

In [28]:
X_binenc = pd.concat([X,categoricals],axis=1)

In [29]:
X_binenc.head()

,production_budget,title_year,aspect_ratio,duration,cast_total_facebook_likes,budget,imdb_score,actor_1_name,director_name
0,425000000.0,2009.000000,1.780000,178.000000,4834.0,2.370000e+08,7.9,CCH Pounder,James Cameron
1,306000000.0,2002.130733,2.126976,108.577186,143.0,4.045539e+07,7.1,Doug Walker,Doug Walker
2,300000000.0,2007.000000,2.350000,169.000000,48350.0,3.000000e+08,7.1,Johnny Depp,Gore Verbinski
3,300000000.0,2015.000000,2.350000,148.000000,11700.0,2.450000e+08,6.8,Christoph Waltz,Sam Mendes
4,275000000.0,2012.000000,2.350000,164.000000,106759.0,2.500000e+08,8.5,Tom Hardy,Christopher Nolan


In [30]:
import category_encoders as ce
encoder = ce.BinaryEncoder(cols=['actor_1_name','director_name'])

In [31]:
encoder.fit_transform(X_binenc).shape

(4104, 31)

In [32]:
X_binenc = encoder.fit_transform(X_binenc)

In [33]:
Xb_train, Xb_test, y_train, y_test = train_test_split(X_binenc,y)

In [34]:
X_train, X_test = (Xb_train[X.columns],Xb_test[X.columns])

In [35]:
model_binenc = Lasso()
model = Lasso()

In [36]:
model_binenc.fit(Xb_train,y_train)
model.fit(X_train,y_train)

Lasso()

In [37]:
print(model_binenc.score(Xb_test,y_test))
print(model.score(X_test,y_test))

0.5758558987567001
0.5788545699842991


Aumentamos el rendimiento de nuestro algoritmo pero no de forma significativa. Mantengamos entonces la dimensionalidad de nuestro espacio de features baja, y vamos a buscar modelos más complejos.

## Conocimiento experto

<div class="alert alert-success">
  Una grán parte del diseño de las features pasa por un **conocimiento espécifico del dominio en el que se esta trabajando**. <br>
  Por ejemplo para analizar una imagen nuestro cerebro no se concentra en los millones de pixeles de una imagen, pero sólo en algunos relevantes como los de los contornos. Durante un buen tiempo **los sistemas de visión de computadores encodeaban features que traducían este conocimiento experto (contornos).** <br>
  Una de las únicas formas de obtener este conocimiento de forma sistemática es ir a bucear en repositorios de papers de Machine Learning como Arxiv, y estudiar la investigación que se ha hecho sobre el dominio específico.
</div>

## Más datos de calidad

Nada le gana conseguir más datos que sean encodeables en features de calidad.

_**Pirámide de Maslow del Machine Learning**_

<img src="https://raw.githubusercontent.com/JuanPabloMF/machine-learning-platzi/8c90f12d2f94ba2b58d725d2eb232a8d654cfc22/img/maslow.png">

Contamos con la base de datos de ganancias de las péliculas el primer fin de semana de exhibición, así como la cantidad de cines en la que fue estrenada.

In [38]:
pd.read_csv('https://raw.githubusercontent.com/JuanPabloMF/datasets-platzi-course/master/datasets/opening_df.csv').head()

,Unnamed: 0,movie_title,opening_gross,screens
0,0,10 Days in a Madhouse,2451.0,10.0
1,1,10 Things I Hate About You,8330681.0,2271.0
2,2,102 Dalmatians,19883351.0,2704.0
3,3,12 Rounds,5329240.0,2331.0
4,4,12 Years a Slave,923715.0,19.0


¿Se puede mejorar considerablemente la predicción?